In [1]:
import pandas as pd
import numpy as np
import copy
import itertools
# Package for Levenstein string distance
import jellyfish
import time

In [2]:
path = "C://Users//Dimitri//Desktop//ENSAE3A//NetworkData//Data//"

In [3]:
attrs = pd.read_csv(path + "attrs_sub.csv")

In [86]:
def authors_parser(authors_string, sep=";"):
    authors = authors_string.split("; ")
    for i in range(0, len(authors)) : 
        if authors[i][0] == " ":
            authors[i] = authors[i][1:]
    return authors

In [87]:
attrs["authors_list"] = attrs["authors"].apply(authors_parser)

In [88]:
def normalized_edit_distance(str1, str2):
    dist = jellyfish.levenshtein_distance(str1, str2)
    return dist / max(len(str1), len(str2))

In [89]:
# Fill all authors
authors = []
for author in attrs["authors_list"]:
    authors += author
# Remove duplicates
authors = list(set(authors))

In [90]:
authors

['Matthijs R. Wildenbeest',
 'Campbell, John Y.',
 'Harrenstein, Paul',
 'J. Anthony Cookson',
 'Erard, Brian',
 'Bradley, John',
 'Anmol Bhandari',
 'Train, Kenneth',
 'William T. Gibb',
 'Dehez, Pierre',
 'Heyes, Anthony',
 'Kim, Joonghyuk',
 'Lawrenz, Jochen',
 'Ong, David',
 'Mika Maliranta',
 'Canner, Niko',
 'Herbert von Beckerath',
 'Stong, Richard',
 'Crocker, Thomas D',
 'Fischer, Sven',
 'Rebelo, Sergio T.',
 'Tuvia Blumenthal',
 'Sheppard, Kevin',
 'Joongho Han',
 'Yan, Jianye',
 'Oren Rigbi',
 'Santini, Massimiliano',
 'Mikael Lindahl',
 'Costas Meghir',
 'D. F. Nicholls',
 'Fowler, Anthony',
 'Charles Blackorby',
 'Prasnikar, Janez',
 'D. A. Collard',
 'Chang, Chong-Chuo',
 'Trejo, Stephen J',
 'Charles M. Tiebout',
 'Norihiko Yamano',
 'Mark W. Watson',
 'James C. Horne',
 'Agulnik, Phil',
 'Purroy, Pedro',
 'Ulubaşoğlu, Mehmet Ali',
 'Sibylle Lehmann-Hasemeyer',
 'Neher, Philip A',
 'George M Korniotis',
 'Mountford, Andrew',
 'Matthew Grennan',
 'Riou, Stéphane',
 'Swam

In [91]:
df_authors = pd.DataFrame(authors, columns=["original"])

In [92]:
df_authors

,original
0,Matthijs R. Wildenbeest
1,"Campbell, John Y."
2,"Harrenstein, Paul"
3,J. Anthony Cookson
4,"Erard, Brian"
5,"Bradley, John"
6,Anmol Bhandari
7,"Train, Kenneth"
8,William T. Gibb
9,"Dehez, Pierre"


In [93]:
def uniformize_names(str1):
    if "," in str1:
        splitted = str1.split(", ")
        if len(splitted) > 1:
            str1bis = splitted[1] + " " + splitted[0]
        else :
            str1bis = str1
        return str1bis
    else:
        return str1

In [16]:
df_authors["uniformat"] = df_authors["original"].apply(uniformize_names)

In [47]:
df_authors_reduced = df_authors.iloc[11000: 12000, :].copy()

In [63]:
def map_authors(auths_df, thresh):
    cleaned_df = pd.DataFrame(columns=["original", "uniformat", "equivalent"])
    for i in auths_df.index:
        author = auths_df.loc[i, "uniformat"]
        author_original = auths_df.loc[i, "original"]
        splitted = author.split(" ")
        last = splitted[-1]
        search = cleaned_df["uniformat"].str.contains(last, regex=False)
        ind_search = search[search == True].index
        first = splitted[0]
        prop = (len(first) == 1) or ((len(first) == 2) and ("." in first))
        exists_similar = False
        for ind in ind_search :
            dist = normalized_edit_distance(cleaned_df["uniformat"][ind], 
                                            author)
            dist_test = dist <= thresh
            if dist_test :
                exists_similar = True
                cleaned_df["equivalent"][ind].append(author_original)    
            # first_letter_test1 = prop and (first[0] == cleaned_df["uniformat"][ind][0])
            # if first_letter_test1:
                # exists_similar = True
                # cleaned_df["equivalent"][ind].append(author_original) 
        if not exists_similar:
            df = pd.DataFrame(columns=["original", "uniformat", "equivalent"], index=[0])
            df.set_value(0, "original", author_original)
            df.set_value(0, "uniformat", author)
            df.set_value(0, "equivalent", [author_original])
            cleaned_df = cleaned_df.append(df, ignore_index=True)
        if i%1000 == 0:
            print(i)
    return cleaned_df

In [64]:
start = time.clock()
cleaned = map_authors(df_authors, 0.1)
end = time.clock()
print(end - start)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
1009.7924934654384


In [125]:
cleaned = cleaned.sort_values(by="uniformat")
cleaned = cleaned.reset_index(drop=True)

In [126]:
cleaned

,original,uniformat,equivalent,alpha_order
0,&Lubos Pástor,&Lubos Pástor,"[&Lubos Pástor, Lubos Pástor]",&
1,"Hao, (Grace) Qing",(Grace) Qing Hao,"[Hao, (Grace) Qing]",H
2,1816-1906,1816-1906,[1816-1906],1
3,"Kalay, A",A Kalay,"[Kalay, A]",K
4,"Auquier, A A",A A Auquier,"[Auquier, A A]",A
5,"Ryvkin, A A",A A Ryvkin,"[Ryvkin, A A]",R
6,"Walters, A A",A A Walters,"[Walters, A A]",W
7,"Abhyankar, A",A Abhyankar,"[Abhyankar, A]",A
8,"Schmid, A Allan",A Allan Schmid,"[Schmid, A Allan]",S
9,"Anastasopoulos, A",A Anastasopoulos,"[Anastasopoulos, A]",A


In [129]:
cleaned.to_csv(path + "authors.csv")

ValueError: labels ['alpha_order'] not contained in axis

In [158]:
def unpack_authors_list(authors_df):
    n_eqs = authors_df["equivalent"].apply(lambda x: len(x))
    max_eqs = n_eqs.max()
    for i in range(0, max_eqs):
        authors_df["equivalent_" + str(i)] = np.nan
    for i in authors_df.index:
        for j in range(0, n_eqs[i]):
            authors_df.set_value(i, "equivalent_" + str(j), authors_df.loc[i, "equivalent"][j])
    return authors_df

In [156]:
cleaned = unpack_authors_list(cleaned)

In [170]:
eqs_cols = ["equivalent_" + str(i) for i in range(0, 7)]

In [172]:
def author_corresp_bis(authors_df, eqs_cols, author_list):
    n_eqs = len(eqs_cols)
    authors_nos = []
    for author in author_list :
        found = False
        i = 0
        while (i <= n_eqs) and (not found):
            search = authors_df[eqs_cols[i]].str.contains(author, regex=False)
            ind_search = search[search == True].index
            if len(ind_search) >= 1:
                found = True
                authors_nos.append(ind_search[0])
            i += 1
    return authors_nos

In [ ]:
start = time.clock()
attrs["authors_nos"] = attrs["authors_list"].apply(lambda x: author_corresp_bis(cleaned, eqs_cols, x))
end = time.clock()
print(end - start)
# attrs.to_csv(path + "attrs_nos.csv")